In [ ]:
import json
import cv2
import matplotlib.pyplot as plt
import sys
import os
import numpy as np # cv2.rectangle and other operations might return numpy arrays

# --- Configuration: Determine Project Root and Adjust Paths/IDs as needed ---

current_executed_dir = os.getcwd()
# print(f"Current executed_dir (os.getcwd()): {current_executed_dir}")

# Determine the project root (which is the 'nutri-snap' folder)
if os.path.basename(current_executed_dir) == 'notebooks' and \
   os.path.basename(os.path.abspath(os.path.join(current_executed_dir, '..'))) == 'nutri-snap':
    project_root = os.path.abspath(os.path.join(current_executed_dir, '..'))
elif os.path.basename(current_executed_dir) == 'nutri-snap':
    project_root = current_executed_dir
else:
    # Fallback or error if the directory structure is not recognized
    print(f"WARNING: Unexpected directory structure. Attempting to use '{current_executed_dir}' as the nutri-snap project root.")
    print("Please run this script from the 'nutri-snap' directory or the 'nutri-snap/notebooks' directory.")
    project_root = current_executed_dir # Default assumption, may require manual adjustment

# print(f"Project root (nutri-snap folder): {project_root}")

if project_root not in sys.path:
    sys.path.append(project_root)

# Now you can import from external_libs
# Make sure the path external_libs/ml_4m/fourm/utils/plotting_utils.py is correct relative to project_root
try:
    from external_libs.ml_4m.fourm.utils.plotting_utils import visualize_bboxes
except ImportError as e:
    print(f"Could not import visualize_bboxes. Make sure the file external_libs/ml_4m/fourm/utils/plotting_utils.py exists and is accessible.")
    print(f"PYTHONPATH includes: {sys.path}")
    print(f"Error details: {e}")
    # As a fallback, you might need to copy visualize_bboxes and convert_string_to_bboxes here
    # For now, we'll let it raise an error if the import fails.
    raise

# Example paths - YOU MUST MODIFY THESE to point to your actual data
SPLIT = 'train' # Or 'test', 'val', etc.
DISH_ID = 'dish_1558640849' # Change to your desired dish ID

# Path to the directory containing processed data
# Assumes project_root is the 'nutri-snap' directory
absolute_base_processed_dir = os.path.join(project_root, 'data', 'processed')
# print(f"Absolute base processed dir: {absolute_base_processed_dir}")

rgb_image_path = os.path.join(absolute_base_processed_dir, SPLIT, 'rgb', DISH_ID, f'{DISH_ID}.png')
bbox_json_path = os.path.join(absolute_base_processed_dir, SPLIT, 'bounding_box', DISH_ID, f'{DISH_ID}.json')

# print(f"Attempting to load RGB image from: {rgb_image_path}")
# print(f"Attempting to load Bbox JSON from: {bbox_json_path}")

# Constants for coordinate conversion (used by visualize_bboxes via convert_string_to_bboxes)
BINS = 1000 # This is the default in convert_string_to_bboxes

# --- Load Image ---
img = cv2.imread(rgb_image_path)
if img is None:
    print(f"Error: Could not load image from {rgb_image_path}")
    # Consider raising an error or exiting if the image is essential
    img_display = np.zeros((224, 224, 3), dtype=np.uint8) # Placeholder
    img_h, img_w = 224, 224 # Placeholder
else:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR (OpenCV default) to RGB for Matplotlib
    img_h, img_w = img.shape[:2]
    img_display = img.copy() # Work on a copy

# --- Load Bounding Box JSON ---
instances = [] # Initialize instances as an empty list
try:
    with open(bbox_json_path, 'r') as f:
        bboxes_data_dict = json.load(f) # Load the entire dictionary
    # Extract the list of instances from the dictionary
    instances = bboxes_data_dict.get('instances', [])
except FileNotFoundError:
    print(f"Warning: Bounding box file not found at {bbox_json_path}. Image will be displayed without boxes.")
    # instances is already initialized to []
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {bbox_json_path}. Image will be displayed without boxes.")
    # instances is already initialized to []

# --- Convert JSON bboxes to string format for visualize_bboxes ---
# The visualize_bboxes function expects a string like: "v0=x1 v1=y1 v2=x2 v3=y2 class_name1 v0=x3 v1=y3 ..."
# The coordinates in the string should be scaled as if they were derived from normalized coordinates multiplied by (BINS - 1).
# Original N5K bbox coordinates are absolute pixel values.
bbox_parts_for_string = []
if instances and img_display is not None: # Only proceed if there are instances and an image to get dimensions from
    for item in instances: # Iterate over each instance dictionary in the 'instances' list
        class_name = item.get('class_name', 'unknown_class') # Use 'class_name'
        bbox_coords_abs = item.get('boxes') # Use 'boxes'. Expected: [x_min_abs, y_min_abs, x_max_abs, y_max_abs]

        if bbox_coords_abs and len(bbox_coords_abs) == 4:
            x_min_abs, y_min_abs, x_max_abs, y_max_abs = bbox_coords_abs

            # Convert absolute pixel coordinates to the scaled format expected in the string
            # X_scaled_for_string = (X_absolute / image_dimension_size) * (BINS - 1)
            # Ensure img_w and img_h are not zero to avoid DivisionByZeroError
            if img_w == 0 or img_h == 0:
                print(f"Error: Image dimensions are zero (width={img_w}, height={img_h}). Cannot calculate box coordinates.")
                continue

            v0_val = (x_min_abs / img_w) * (BINS - 1)
            v1_val = (y_min_abs / img_h) * (BINS - 1)
            v2_val = (x_max_abs / img_w) * (BINS - 1)
            v3_val = (y_max_abs / img_h) * (BINS - 1)
            
            # The class name can contain spaces. visualize_bboxes handles this.
            bbox_parts_for_string.append(f"v0={int(round(v0_val))} v1={int(round(v1_val))} v2={int(round(v2_val))} v3={int(round(v3_val))} {class_name}")
        else:
            print(f"Warning: Invalid or missing bbox data for item: {item}")

bboxes_str_for_viz = " ".join(bbox_parts_for_string)

# --- Visualize ---
# visualize_bboxes expects an RGB numpy array.
# It returns the image with bounding boxes drawn on it.
if img_display is not None: # Check if an image (original or placeholder) is available
    # Ensure visualize_bboxes is successfully imported
    if 'visualize_bboxes' in globals() or 'visualize_bboxes' in locals():
        # Pass a copy of the image if you don't want the original 'img_display' array to be modified by visualize_bboxes
        img_with_bboxes = visualize_bboxes(img_display.copy(), bboxes_str_for_viz)

        plt.figure(figsize=(10, 10))
        plt.imshow(img_with_bboxes)
        plt.axis('off')
        plt.title(f"Bounding Boxes for {DISH_ID} (Split: {SPLIT})")
        plt.show()
    else:
        print("Error: The 'visualize_bboxes' function was not imported correctly. Visualization impossible.")
        # Fallback: show image without bboxes if visualize_bboxes is missing
        plt.figure(figsize=(10, 10))
        plt.imshow(img_display) # Show original or placeholder image
        plt.axis('off')
        plt.title(f"Image {DISH_ID} (visualize_bboxes not available)")
        plt.show()
else:
    print("Image not loaded, cannot visualize.")